In [ ]:
pip install scikit-learn

In [104]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [105]:
houses = pd.read_csv("Housing.csv")

Y = houses['price'].values.reshape(-1, 1)
X = houses[['area', 'bedrooms', 'bathrooms']].values

Функція гіпотези лінійної регресії у векторному вигляді:
$$ h(\vec{x}) = \vec{x} \cdot \vec{w} $$
$$ \vec{x} = \begin{pmatrix} 1 \\ x_1 \\ x_2 \\ ... \\ x_i \end{pmatrix} 
\quad
\vec{w} = \begin{pmatrix}  w_1 \\ w_2 \\ ... \\ w_{i-1} \\ w_i \end{pmatrix} $$

In [106]:
def linear_regression_hypothesis(w0, w, x):
    return w0 + np.dot(x, w)

Функція для обчислення функції втрат (MSE) у векторному вигляді:
$$ MSE = \frac{1}{2m} \sum_{i=1}^m (h(x^i) - y^i)^2 $$

In [147]:
def mean_squared_error(y_true, y_pred, m):
    return (1 / (2 * m)) * np.sum((y_true - y_pred) ** 2)

Градієнтний спуск:

In [239]:
from sklearn.preprocessing import StandardScaler

def grad_descent(x, y, iter=1000, lr=0.000000001, e=1e-4):
    scaler_x = StandardScaler()
    x_scaled = scaler_x.fit_transform(x)
    
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))
    
    intercept = 0
    slope = np.zeros(x.shape[1]).reshape(-1, 1)

    n = len(y)
    prev_cost = None

    for i in range(iter):
        y_pred = linear_regression_hypothesis(intercept, slope, x_scaled)
        cost = mean_squared_error(y_scaled, y_pred, n)

        if prev_cost and abs(cost - prev_cost) <= e:
            print(f"Градієнтний спуск завершено на {i+1}-й ітерації.")
            break

        prev_cost = cost
        
        slope_derivative = (1/n) * np.dot(x_scaled.T, (y - y_pred))
        intercept_derivative = (1/n) * np.sum(y_scaled - y_pred)
        
        slope += lr * slope_derivative
        intercept += lr * intercept_derivative

        # print(f"Ітерація {i+1}: Вартість = {cost:.4f}, Slope = {slope.T}, Intercept = {intercept}")

        if cost > 1e5:
            print("Зупинка: Вартість перевищує межу.")
            break
    slope = slope / scaler_x.scale_.reshape(-1, 1)
    intercept = scaler_y.inverse_transform([[intercept - np.sum(slope.T * scaler_x.mean_)]])[0, 0]

    return slope, intercept


In [240]:
slope, intercept = grad_descent(X, Y)
print(f"Slope = {slope.T}, Intercept = {intercept}")

Градієнтний спуск завершено на 343-й ітерації.
Slope = [[1.57995379e-04 3.17645442e-01 6.58882976e-01]], Intercept = -97752.19473536313


Аналітичне рішення для параметрів лінійної регресії:
$$w = (X^T \cdot X)^{-1} \cdot X^T \cdot y$$

In [243]:
X_scaled = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1).astype("int64")

weights = np.linalg.inv(X_scaled.T @ X_scaled) @ X_scaled.T @ Y
print(f"Slope = {weights[1:].T}, Intercept = {weights[0]}")

Slope = [[3.78762754e+02 4.06820034e+05 1.38604950e+06]], Intercept = [-173171.60763264]


Перевірка спрогнозованих значень:

In [244]:
lin_reg = LinearRegression()
lin_reg.fit(X, Y)
print(f"Slope = {lin_reg.coef_}, Intercept = {lin_reg.intercept_}")

Slope = [[3.78762754e+02 4.06820034e+05 1.38604950e+06]], Intercept = [-173171.60763264]
